# Coursework 2 for Cardiac MR Image Segmentation (2020-2021)

After you have gone through the coursework description, this tutorial is designed to further helps you understand the problem and therefore enable you to propose a good solution for this coursework. You will learn:

* how to load and save images with OpenCV
* how to train a segmentation model with Pytorch
* how to evaluate the trained model

## 1. Load, show, and save images with OpenCV

OpenCV is an open-source computer vision library which helps us to manipulate image data. In this section, we will cover:
* Loading an image from file with imread()
* Displaying the image with matplotlib plt.imshow()
* Saving an image with imwrite()

For a more comprehensive study of OpenCV, we encourage you to check the official [OpenCV documentation](https://docs.opencv.org/master/index.html).

In [ ]:
from matplotlib import pyplot as plt
def show_image_mask(img, mask, cmap='gray'): # visualisation
    fig = plt.figure(figsize=(5,5))
    plt.subplot(1, 2, 1)
    plt.imshow(img, cmap=cmap)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap=cmap)
    plt.axis('off')

In [ ]:
import os
import cv2 #import OpenCV

data_dir = './data/train'
image = cv2.imread(os.path.join(data_dir,'image','cmr1.png'), cv2.IMREAD_UNCHANGED)
mask = cv2.imread(os.path.join(data_dir,'mask','cmr1_mask.png'), cv2.IMREAD_UNCHANGED)
show_image_mask(image, mask, cmap='gray')
plt.pause(1)
cv2.imwrite(os.path.join('./','cmr1.png'), mask*85)

Note: You will no doubt notice that the mask images appear to be completely black with no sign of any segmentations. This is because the max intensity of pixels in an 8-bit png image is 255 and your image viewer software only sees 255 as white. For those values close to zero, you will only see dark values. This is the case for our masks as the background, the right ventricle, the myocardium, and the left ventricle in each image are 0, 1, 2, and 3, respectively. All of which are close to zero. If we multiply the original mask by 85 and save the result to the directory where this code is, we can see the heart indeed shows up. 

## 2. Define a segmentation model with Pytorch

In this section, we expect you to learn how to:
* Define a Segmentation Model
* Define a DataLoader that inputs images to the Model
* Define training parameters and train the model
* Test the trained model with a new input image

### 2.1 Define a DataLoader

Below we provide you with a dataloader to use in your assigment. You will only need to focus on the development of your model and loss function.



In [ ]:
import torch
import torch.utils.data as data
import cv2
import os
from glob import glob

class TrainDataset(data.Dataset):
    def __init__(self, root=''):
        super(TrainDataset, self).__init__()
        self.img_files = glob(os.path.join(root,'image','*.png'))
        self.mask_files = []
        for img_path in self.img_files:
            basename = os.path.basename(img_path)
            self.mask_files.append(os.path.join(root,'mask',basename[:-4]+'_mask.png'))
            

    def __getitem__(self, index):
            img_path = self.img_files[index]
            mask_path = self.mask_files[index]
            data = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            label = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
            return torch.from_numpy(data).float(), torch.from_numpy(label).float()

    def __len__(self):
        return len(self.img_files)

class TestDataset(data.Dataset):
    def __init__(self, root=''):
        super(TestDataset, self).__init__()
        self.img_files = glob(os.path.join(root,'image','*.png'))

    def __getitem__(self, index):
            img_path = self.img_files[index]
            data = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            return torch.from_numpy(data).float()

    def __len__(self):
        return len(self.img_files)


### 2.2 Define a Segmenatation Model

You will need to define your CNN model for segmentation below

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ContBatchNorm3d(nn.modules.batchnorm._BatchNorm):

    def forward(self, input):
        return F.batch_norm(
            input, self.running_mean, self.running_var, self.weight, self.bias,
            True, self.momentum, self.eps)


class LUConv(nn.Module):
    def __init__(self, in_chan, out_chan, act):
        super(LUConv, self).__init__()
        self.conv1 = nn.Conv2d(in_chan, out_chan, kernel_size=3, padding=1)
        self.bn1 = ContBatchNorm3d(out_chan)

        if act == 'ReLU':
            self.activation = nn.ReLU()
        elif act == 'PReLU':
            self.activation = nn.PReLU()
        elif act == 'LeakyReLU':
            self.activation = nn.LeakyReLU()
        elif act == 'ELU':
            self.activation = nn.ELU(inplace=True)
        elif act == 'Tanh':
            self.activation = nn.Tanh()
        else:
            raise

    def forward(self, x):
        out = self.activation(self.bn1(self.conv1(x)))
        return out


def _make_nConv(in_channel, depth, act, double_chnnel=False):
    if double_chnnel:
        layer1 = LUConv(in_channel, 32 * (2 ** (depth+1)),act)
        layer2 = LUConv(32 * (2 ** (depth+1)), 32 * (2 ** (depth+1)),act)
    else:
        layer1 = LUConv(in_channel, 32*(2**depth),act)
        layer2 = LUConv(32*(2**depth), 32*(2**depth)*2,act)

    return nn.Sequential(layer1,layer2)


class DownTransition(nn.Module):
    def __init__(self, in_channel,depth, act):
        super(DownTransition, self).__init__()
        self.ops = _make_nConv(in_channel, depth,act)
        self.maxpool = nn.MaxPool2d(2)
        self.current_depth = depth

    def forward(self, x):
        if self.current_depth == 3:
            out = self.ops(x)
            out_before_pool = out
        else:
            out_before_pool = self.ops(x)
            out = self.maxpool(out_before_pool)
        return out, out_before_pool

    
class UpTransition(nn.Module):
    def __init__(self, inChans, outChans, depth,act):
        super(UpTransition, self).__init__()
        self.depth = depth
        self.up_conv = nn.ConvTranspose2d(inChans, outChans, kernel_size=2, stride=2)
        self.ops = _make_nConv(inChans+ outChans//2,depth, act, double_chnnel=True)

    def forward(self, x, skip_x):
        out_up_conv = self.up_conv(x)
        concat = torch.cat((out_up_conv,skip_x),1)
        out = self.ops(concat)
        return out


class OutputTransition(nn.Module):
    def __init__(self, inChans, n_labels):

        super(OutputTransition, self).__init__()
        self.final_conv = nn.Conv2d(inChans, n_labels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.sigmoid(self.final_conv(x))
        return out

    
class CNNSEG(nn.Module): # Define your model
    def __init__(self, n_class=4, act='ReLU'):
        super(CNNSEG, self).__init__()
        # fill in the constructor for your model here
        self.down_tr64 = DownTransition(1,0,act)
        self.down_tr128 = DownTransition(64,1,act)
        self.down_tr256 = DownTransition(128,2,act)
        self.down_tr512 = DownTransition(256,3,act)

        self.up_tr256 = UpTransition(512, 512,2,act)
        self.up_tr128 = UpTransition(256,256, 1,act)
        self.up_tr64 = UpTransition(128,128,0,act)
        self.out_tr = OutputTransition(64, n_class)
        
    def forward(self, x):
        # fill in the forward function for your model here
        self.out64, self.skip_out64 = self.down_tr64(x)
        self.out128,self.skip_out128 = self.down_tr128(self.out64)
        self.out256,self.skip_out256 = self.down_tr256(self.out128)
        self.out512,self.skip_out512 = self.down_tr512(self.out256)

        self.out_up_256 = self.up_tr256(self.out512,self.skip_out256)
        self.out_up_128 = self.up_tr128(self.out_up_256, self.skip_out128)
        self.out_up_64 = self.up_tr64(self.out_up_128, self.skip_out64)
        self.out = self.out_tr(self.out_up_64)

        return self.out

model = CNNSEG() # We can now create a model using your defined segmentation model

### 2.3 Define a Loss function and optimizer

You will need to define a loss function and an optimizer. torch.nn has a variety of readymade loss functions, although you may wish to create your own instead. torch.optim has a variety of optimizers, it is advised that you use one of these.

In [ ]:
"""
Loss function & optimizer
"""
import torch.optim as optim


def sum_tensor(inp, axes, keepdim=False):
    # copy from: https://github.com/MIC-DKFZ/nnUNet/blob/master/nnunet/utilities/tensor_utilities.py
    axes = np.unique(axes).astype(int)
    if keepdim:
        for ax in axes:
            inp = inp.sum(int(ax), keepdim=True)
    else:
        for ax in sorted(axes, reverse=True):
            inp = inp.sum(int(ax))
    return inp

def get_tp_fp_fn(net_output, gt, axes=None, mask=None, square=False):
    """
    net_output must be (b, c, x, y(, z)))
    gt must be a label map (shape (b, 1, x, y(, z)) OR shape (b, x, y(, z))) or one hot encoding (b, c, x, y(, z))
    if mask is provided it must have shape (b, 1, x, y(, z)))
    :param net_output:
    :param gt:
    :param axes:
    :param mask: mask must be 1 for valid pixels and 0 for invalid pixels
    :param square: if True then fp, tp and fn will be squared before summation
    :return:
    """
    if axes is None:
        axes = tuple(range(2, len(net_output.size())))

    shp_x = net_output.shape
    shp_y = gt.shape

    with torch.no_grad():
        if len(shp_x) != len(shp_y):
            gt = gt.view((shp_y[0], 1, *shp_y[1:]))

        if all([i == j for i, j in zip(net_output.shape, gt.shape)]):
            # if this is the case then gt is probably already a one hot encoding
            y_onehot = gt
        else:
            gt = gt.long()
            y_onehot = torch.zeros(shp_x)
            if net_output.device.type == "cuda":
                y_onehot = y_onehot.cuda(net_output.device.index)
            y_onehot.scatter_(1, gt, 1)

    tp = net_output * y_onehot
    fp = net_output * (1 - y_onehot)
    fn = (1 - net_output) * y_onehot

    if mask is not None:
        tp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(tp, dim=1)), dim=1)
        fp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fp, dim=1)), dim=1)
        fn = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fn, dim=1)), dim=1)

    if square:
        tp = tp ** 2
        fp = fp ** 2
        fn = fn ** 2

    tp = sum_tensor(tp, axes, keepdim=False)
    fp = sum_tensor(fp, axes, keepdim=False)
    fn = sum_tensor(fn, axes, keepdim=False)

    return tp, fp, fn

class TverskyLoss(nn.Module):
    def __init__(self, apply_nonlin=None, batch_dice=False, do_bg=True, smooth=1.,
                 square=False):
        """
        paper: https://arxiv.org/pdf/1706.05721.pdf
        """
        super(TverskyLoss, self).__init__()

        self.square = square
        self.do_bg = do_bg
        self.batch_dice = batch_dice
        self.apply_nonlin = apply_nonlin
        self.smooth = smooth
        self.alpha = 0.61
        self.beta = 0.39

    def forward(self, x, y, loss_mask=None):
        shp_x = x.shape

        if self.batch_dice:
            axes = [0] + list(range(2, len(shp_x)))
        else:
            axes = list(range(2, len(shp_x)))

        if self.apply_nonlin is not None:
            x = self.apply_nonlin(x)

        tp, fp, fn = get_tp_fp_fn(x, y, axes, loss_mask, self.square)


        tversky = (tp + self.smooth) / (tp + self.alpha*fp + self.beta*fn + self.smooth)

        if not self.do_bg:
            if self.batch_dice:
                tversky = tversky[1:]
            else:
                tversky = tversky[:, 1:]
        tversky = tversky.mean()

        return 1-tversky

    
def get_loss_func(loss_name):
    if loss_name == "Tversky":
        return TverskyLoss()
    if loss_name == "CrossEntropy":
        return nn.CrossEntropyLoss()

def get_optimizer(model, method, learning_rate):
    if method == "SGD":
        return optim.SGD(model.parameters(), lr=learning_rate)
    elif method == "Adam":
        return optim.AdamW(model.parameters(), lr=learning_rate)
    elif method == "AdaGrad":
        return optim.Adagrad(model.parameters(), lr=learning_rate)

### 2.4 Training

As most of you will use CPUs to train the model, expect your models to take **30 minutes to train if not longer depending on network architecture**. To save time, you should not be using all training data until your model is well developed. If you are running your model on a GPU training should be significantly faster. During the training process, you may want to save the checkpoints as follows:

```
# Saving checkpoints for validation/testing
torch.save(model.state_dict(), path)
```
The saved checkpoints can be used to load at a later date for validation and testing. Here we give some example code for training a model. Note that you need to specify the max iterations you want to train the model.

In [ ]:
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

def train(model, num_workers = 0, batch_size = 4,epochs = 25,learning_rate = 1, optimizer_name = "Adam", loss_name="Tversky", target=None):

    # Get Data Loader for Training Set
    data_path = './data/train'
    train_set = TrainDataset(data_path)
    training_data_loader = DataLoader(dataset=train_set, num_workers=num_workers, batch_size=batch_size, 
                                      shuffle=True,drop_last = True)

    # Get Loss Function and Optimizer
    loss_func = get_loss_func(loss_name)
    optimizer = get_optimizer(model, optimizer_name, learning_rate)

    # Initializes empty containers
    loss_list_train = list()
    loss_list_val = list()
    dice_list_train = list()
    dice_list_val = list()
    
    # Train the Model
    # Epoch loop
    for e in range(epochs):
        iteration_dice_list = list()
        
        # Iteration loop
        for iteration, sample in enumerate(training_data_loader):
            print("Epoch, Iteration:", e, iteration)
            
            # Fetch images and labels.  
            img, mask = sample
            
            # show_image_mask(img[0,...].squeeze(), mask[0,...].squeeze()) # visualise all data in training set
            # plt.pause(1)

            
            # Write your FORWARD below
            # Note: Input image to your model and ouput the predicted mask and Your predicted mask should have 4 channels
            model.train()                                          # Set the model to training mode
            optimizer.zero_grad()                                  # Avoid accumulating gradients
            img = img / 255 - 0.5                                  # Normalization
            y_pred = model(img.reshape(batch_size,1,96,96))        # Forward pass
            
            
            # Then write your BACKWARD & OPTIMIZE below
            # Note: Compute Loss and Optimize
            loss = loss_func(y_pred , mask.long())                 # Compute Loss
            loss.backward()                                        # Calculate Gradients
            optimizer.step()                                       # Update Model's Params
            
            masks_pred = torch.argmax(y_pred, dim=1)               # Merge four channels into one channel
            iteration_dice = see_dice(mask.numpy(), masks_pred.squeeze().numpy())    # Compute Dice
            
            # Record the Loss and Dice of training set for each iteration
            iteration_dice_list.append(iteration_dice)
            loss_list_train.append(loss)
            print("Iteration Loss:",loss)
        
        # Record the Dice of training set for each epoch (Take the mean dice of each iteration)
        epoch_dice = np.mean(iteration_dice_list)
        dice_list_train.append(epoch_dice)
        print("Epoch Dice:", epoch_dice)
        
        # Evaluate by validation set
        val_dice, val_loss = evaluate(model, loss_func)
        loss_list_val.append(val_loss)
        dice_list_val.append(val_dice)
        
        # Check the validation dice with target dice
        if target:
            if val_dice > target:                                          # Find a desirable dice
                model_params_path = "net_params.pth"
                torch.save(model.state_dict(), model_params_path)          # Save the model
                print("The model parameters have been saved.")
                break
    
    # Plot Loss and Dice both in Training set and Validation set
    fig, axs = plt.subplots(nrows = 1, ncols = 2, sharex=False, sharey=False, figsize=(12, 5))
    # Plot Loss
    axs[0].plot(loss_list_train, "b-", label=u"Train Loss") 
    X = (np.arange(len(loss_list_val))+1) * int(100/batch_size)
    axs[0].plot(X, loss_list_val, "ro-", label=u"Val Loss") 
    axs[0].set_xlabel('Iteration')
    axs[0].set_ylabel('Loss')
    axs[0].legend(loc="upper right")
    # Plot Dice 
    axs[1].plot(dice_list_train, "bo-", label=u"Train Dice") 
    for x,y in enumerate(dice_list_train):
        if x % max(int(len(dice_list_train)/5),1) == 0 or x == len(dice_list_train)-1:
            axs[1].text(x, y, '%.3f' % y, fontdict={'fontsize':10})
    axs[1].plot(dice_list_val, "ro-", label=u"Val Dice") 
    for x,y in enumerate(dice_list_val):
        if x % max(int(len(dice_list_val)/5),1) == 0 or x == len(dice_list_val)-1:
            axs[1].text(x, y, '%.3f' % y, fontdict={'fontsize':10})
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('Dice')
    axs[1].legend(loc="lower right")
    plt.suptitle('Comparing Loss and Dice', fontsize=16)
    plt.show()
    
    return dice_list_val[-1], loss_list_val[-1]

### 2.5 Testing

When validating the trained checkpoints (models), remember to change the model status as **Evaluation Mode**

In [ ]:
import numpy as np
from torch.autograd import Variable

In [ ]:
# In this block you are expected to write code to load saved model and deploy it to all data in test set to 
# produce segmentation masks in png images valued 0,1,2,3, which will be used for the submission to Kaggle.

def test(model):
    
    # Get Data Loader for Test Set
    data_path = './data/test'
    num_workers = 0
    batch_size = 2
    test_set = TestDataset(data_path)
    test_data_loader = DataLoader(dataset=test_set, num_workers=num_workers,batch_size=batch_size, shuffle=False)
    
    # Setting
    i_png = 120
    folder = './test_results'
    if not os.path.exists(folder):
        os.mkdir(folder)
    
    # Load saved model
    model_params_path = "net_params.pth"
    model.load_state_dict(torch.load(model_params_path))

    # Test Model
    with torch.no_grad():               # fix the model's params
        model.eval()                    # set model to evaluation mode

        for iteration, sample in enumerate(test_data_loader):
            img = sample                                          # Fetch images and labels

            # plt.imshow(img[0,...].squeeze(), cmap='gray')       # visualise all images in test set
            # plt.pause(1)

            # Produce Segmentation Masks
            img = img / 255 - 0.5                                 # Normalization
            y_pred = model(img.reshape(batch_size,1,96,96))       # Forward pass
            masks = torch.argmax(y_pred, dim=1)                   # Merge four channels into one channel

            # Save png images
            masks = masks.squeeze()
            for mask in masks:
                i_png = i_png + 1
                cv2.imwrite(os.path.join(folder,'cmr%s_mask.png' % i_png), mask.numpy())
    
    print("Saved image successfully")
    return folder

## 3. Evaluation

As we will automatically evaluate your predicted test makes on Kaggle, in this section we expect you to learn:
* what is the Dice score used on Kaggle to measure your models performance
* how to submit your predicted masks to Kaggle

### 3.1 Dice Score

To evaluate the quality of the predicted masks, the Dice score is adopted. Dice score on two masks A and B is defined as the intersection ratio between the overlap area and the average area of two masks. A higher Dice suggests a better registration.

$Dice (A, B)= \frac{2|A \cap B|}{|A| + |B|} $

However, in our coursework, we have three labels in each mask, we will compute the Dice score for each label and then average the three of them as the final score. Below we have given you `categorical_dice` for free so you can test your results before submission to Kaggle.

In [ ]:
def categorical_dice(mask1, mask2, label_class=1):
    """
    Dice score of a specified class between two volumes of label masks.
    (classes are encoded but by label class number not one-hot )
    Note: stacks of 2D slices are considered volumes.

    Args:
        mask1: N label masks, numpy array shaped (H, W, N)
        mask2: N label masks, numpy array shaped (H, W, N)
        label_class: the class over which to calculate dice scores

    Returns:
        volume_dice
    """
    mask1_pos = (mask1 == label_class).astype(np.float32)
    mask2_pos = (mask2 == label_class).astype(np.float32)
    dice = 2 * np.sum(mask1_pos * mask2_pos) / (np.sum(mask1_pos) + np.sum(mask2_pos))
    return dice

In [ ]:
# Compute the dice between two masks in 3 labels
def see_dice(mask1, mask2):
    dice_list = list()
    for i in range(1, 4):
        dice = categorical_dice(mask1, mask2, label_class=i)
        dice_list.append(dice)
    return np.mean(dice_list)

In [ ]:
def evaluate(model, loss_func, num_workers = 0, imshow=False):
    
    # Get Data Loader for Validation Set
    data_path = './data/val'
    val_set = TrainDataset(data_path)
    val_data_loader = DataLoader(dataset=val_set, num_workers=num_workers, shuffle=True)
    
    # Evaluate the model
    with torch.no_grad():               # fix the model's params
        model.eval()                    # set model to evaluation mode
        batch_dice_list = list()
        for iteration, sample in enumerate(val_data_loader):
            img, mask = sample                                   # Fetch images and labels
            img = img/255 - 0.5                                  # Normalization
            y_pred = model(img.reshape(1,1,96,96))               # Forward pass
            val_loss = loss_func(y_pred , mask.long())           # Compute validation loss
            masks = torch.argmax(y_pred, dim=1)                  # Merge four channels into one channel
            
            # Plot the true mask and the predicted mask
            if imshow:
                fig, axs = plt.subplots(nrows = 1, ncols = 2, sharex=True, sharey=True, figsize=(12, 5))
                axs[0].set_title("True Mask")
                axs[1].set_title("Predicted Mask")
                axs[0].imshow(mask.squeeze(), cmap='gray')
                axs[1].imshow(masks.squeeze(), cmap='gray')
                plt.show()
            
            # Record the dice for each iteration
            batch_dice = see_dice(mask.numpy(), masks.squeeze().numpy())       # Compute Dice
            batch_dice_list.append(batch_dice)
            print("Batch Dice:",batch_dice)

        # Record the dice of the entire validation set
        val_dice = np.mean(batch_dice_list)
        print("Validation Dice:",val_dice)
        
        return val_dice, val_loss

### 3.2 Submission

Kaggle requires your submission to be in a specific CSV format. To help ensure your submissions are in the correct format, we have provided some helper functions to do this for you. For those interested, the png images are run-length encoded and saved in a CSV to the specifications required by our competition.

It is sufficient to use this helper function. To do so, save your 80 predicted masks into a directory. ONLY the 80 predicted masks should be in this directory. Call the submission_converter function with the first argument as the directory containing your masks, and the second the directory in which you wish to save your submission.

In [ ]:
import numpy as np
import os
import cv2

def rle_encoding(x):
    '''
    *** Credit to https://www.kaggle.com/rakhlin/fast-run-length-encoding-python ***
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev + 1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def submission_converter(mask_directory, path_to_save):
    writer = open(os.path.join(path_to_save, "submission.csv"), 'w')
    writer.write('id,encoding\n')

    files = os.listdir(mask_directory)

    for file in files:
        name = file[:-4]
        mask = cv2.imread(os.path.join(mask_directory, file), cv2.IMREAD_UNCHANGED)

        mask1 = (mask == 1)
        mask2 = (mask == 2)
        mask3 = (mask == 3)

        encoded_mask1 = rle_encoding(mask1)
        encoded_mask1 = ' '.join(str(e) for e in encoded_mask1)
        encoded_mask2 = rle_encoding(mask2)
        encoded_mask2 = ' '.join(str(e) for e in encoded_mask2)
        encoded_mask3 = rle_encoding(mask3)
        encoded_mask3 = ' '.join(str(e) for e in encoded_mask3)

        writer.write(name + '1,' + encoded_mask1 + "\n")
        writer.write(name + '2,' + encoded_mask2 + "\n")
        writer.write(name + '3,' + encoded_mask3 + "\n")

    writer.close()

## 4. Experiments for Different Hyperparameters

In [ ]:
import csv

"""
Choose Different Hyperparameters
"""
# loss_name = 'CrossEntropy'
loss_name = 'Tversky'

# optimizer_name = 'SGD'
optimizer_name = 'Adam'
# optimizer_name = 'AdaGrad'

activation_name = 'ReLU'
# activation_name = 'Tanh'
# activation_name = 'LeakyReLU'

lr_search_list = [1,1e-1,1e-2,1e-3,1e-4,1e-5,1e-6]
batch_search_list = [1,2,4,8,16,32,64]
epochs = 25
num_workers = 0
    
# Record the result into a CSV file
file_name = activation_name + "_" + loss_name + "_" + optimizer_name + '.csv'

"""
Do Experiments with Different Hyperparameters
"""
with open(file_name, 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Learning Rate", "Batch Size","Epochs","Loss Function", "Optimizer","Activation","Mean Dice","Validation Loss"])
    
    for lr in lr_search_list:
        print("Test for Learning Rate: ", lr)
        for batch in batch_search_list:
            print("Test for Batch Size: ", batch)
            
            # Initialize the model
            model = CNNSEG(act=activation_name)
            
            # Train the model and get the validation result
            meanDice, valLoss = train(
                model = model,
                num_workers = num_workers, 
                batch_size = batch,
                epochs = epochs,
                learning_rate = lr, 
                optimizer_name = optimizer_name,
                loss_name = loss_name
            )
            
            # Record the validation result 
            writer.writerow([lr, batch, epochs, loss_name, optimizer_name, activation_name, meanDice, valLoss])

## 5. Generate the Final Model to Submit

In [ ]:
# Step 1 - Set Best Hyperparameters
loss_name = 'Tversky'
optimizer_name = 'Adam'
activation_name = 'ReLU'
learning_rate = 0.0001
batch_size = 1
epochs = 25
num_workers = 0
model = CNNSEG(act=activation_name)

In [ ]:
# Step 2 - Train the model
model_para_path = train(
    model = model, 
    num_workers = num_workers, 
    batch_size = batch_size,
    epochs = epochs,
    learning_rate = learning_rate, 
    optimizer_name = optimizer_name, 
    loss_name= loss_name, 
    target=0.85
)

In [ ]:
# Step 3 - Visualise prediction in Validation Set
model_params_path = "net_params.pth"
model.load_state_dict(torch.load(model_params_path))
evaluate(model, get_loss_func(loss_name), imshow=True)

In [ ]:
# Step 4 - Implement on Test Set
folder = test(model)

# Generate CSV file
submission_converter(folder, './')
print("Generate CSV file successfully")